In [ ]:
pip install mygrads

In [ ]:
import sys
sys.path.append('C:\\Users\\Gabriel\\GitHub\\bloqueios')
import newBlock as nb
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.patches as mpatches
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.util import add_cyclic_point
from datetime import datetime
import os
import seaborn as sns
import metpy.calc as mpcalc
import re
import calendar
from matplotlib.offsetbox import AnchoredText
import mygrads as mg
sns.set()

In [ ]:
teste = nb.dataset('D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/ZCAS/multiVarsSingleLevel.nc','ERA-5')
teste.dataset

In [ ]:
teste2 = nb.dataset('D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/ZCAS/multiVarsZCAS.nc','ERA-5')
t2 = teste2.dataset.sel(level=200)

In [ ]:
t2.sel(time=slice('1979-12-04','2009-12-09')).mean(axis=0).uwnd

In [ ]:
mg.hdivg(t2.sel(time=slice('1979-12-04','1979-12-09')).mean(axis=0).uwnd.values, t2.sel(time=slice('1979-12-04','1979-12-09')).mean(axis=0).vwnd.values, t2.lat.values, t2.lon.values) *10**5 

In [ ]:
teste2.dataset

In [ ]:
t.sel(time=slice('1979-12-04','1979-12-09')).mean(axis =0)

# Setting Data

In [ ]:
tempoAnalise = ['2009-12-04','2009-12-09']
tempoBase = ['1979-12-04','2019-12-09']

In [ ]:
def data(var, level, tempo):
    if level == None:
        dados = nb.dataset('D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/ZCAS/multiVarsSingleLevel.nc','ERA-5')
    else:
        dados = nb.dataset('D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/ZCAS/multiVarsZCAS.nc', 'ERA-5')
        dados = dados.formatar(level = level)
    uwnd = dados.dataset.sel(time=slice(tempo[0],tempo[1])).mean(axis=0).uwnd.values
    vwnd = dados.dataset.sel(time=slice(tempo[0],tempo[1])).mean(axis=0).vwnd.values
    
    if var == 'hgt':
        dados = dados.dataset.hgt / 9.81
        dados = dados.sel(time=slice(tempo[0],tempo[1])).mean(axis=0)
    elif var == 'uwnd':
        dados = dados.dataset.uwnd
        dados = dados.sel(time=slice(tempo[0],tempo[1])).mean(axis=0)
    elif var == 'vwnd':
        dados = dados.dataset.vwnd
        dados = dados.sel(time=slice(tempo[0],tempo[1])).mean(axis=0)
    elif var == 'w':
        dados = dados.dataset.w
        dados = dados.sel(time=slice(tempo[0],tempo[1])).mean(axis=0)
    elif var == 'msl':
        dados = dados.dataset.msl
        dados = dados.sel(time=slice(tempo[0],tempo[1])).mean(axis=0)
    elif var == 'ttr':
        dados = dados.dataset.ttr
        dados = dados.sel(time=slice(tempo[0],tempo[1])).mean(axis=0)
    
    lons = dados.lon.values
    lats = dados.lat.values
    
    return lons, lats, dados, uwnd, vwnd

In [ ]:
def div(var, level, tempo):
    dados = nb.dataset('D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/ZCAS/multiVarsZCAS.nc', 'ERA-5')
    dados2 = dados.formatar(level = level)
    uwnd = dados2.dataset.sel(time=slice(tempo[0],tempo[1])).mean(axis=0).uwnd.values
    vwnd = dados2.dataset.sel(time=slice(tempo[0],tempo[1])).mean(axis=0).vwnd.values
   
    #dados3 = dados2.dataset.sel(time=slice(tempo[0],tempo[1])).mean(axis=0)
    
    div = mg.hdivg(uwnd, vwnd, dados2.dataset.lat.values, dados2.dataset.lon.values) *10**5 
    
    lons = dados2.dataset.lon.values
    lats = dados2.dataset.lat.values
    
    return lons, lats, div, uwnd, vwnd

In [ ]:
def vort(var, level, tempo):
    dados = nb.dataset('D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/ZCAS/multiVarsZCAS.nc', 'ERA-5')
    dados2 = dados.formatar(level = level)
    uwnd = dados2.dataset.sel(time=slice(tempo[0],tempo[1])).mean(axis=0).uwnd.values
    vwnd = dados2.dataset.sel(time=slice(tempo[0],tempo[1])).mean(axis=0).vwnd.values
   
    #dados3 = dados2.dataset.sel(time=slice(tempo[0],tempo[1])).mean(axis=0)
    
    vort = mg.hcurl(uwnd, vwnd, dados2.dataset.lat.values, dados2.dataset.lon.values) *10**5 
    
    lons = dados2.dataset.lon.values
    lats = dados2.dataset.lat.values
    
    return lons, lats, vort, uwnd, vwnd

In [ ]:
def figuraComp(var, level, periodoBase, periodo, title):
    fig, ax = plt.subplots(nrows=1, ncols=2, subplot_kw=dict(projection = ccrs.PlateCarree()), figsize=(32,26))
    fig.suptitle(title, style ='oblique', fontsize=34, weight='bold', y = 0.72, x = 0.505)
    
    plotComp(ax[0], var, level, periodoBase)
    plotComp(ax[1], var, level, periodo)
    

In [ ]:
def plotComp(ax, var, level, tempo):
    if var == 'div':
        lons, lats, valor, uwnd, vwnd = div(var, level, tempo)
    elif var == 'vort':
        lons, lats, valor, uwnd, vwnd = vort(var, level, tempo)
    else:
        lons, lats, valor, uwnd, vwnd = data(var, level, tempo)
    
    #clevs = np.arange(11000, 13000, 150)
    cf = ax.contourf(lons, lats, valor, extend='both', cmap = 'Spectral_r', transform=ccrs.PlateCarree())
    if level != None:
        x = lons[::10]
        y = lats[::10]
        #brb = ax.barbs(x, y, uwnd, vwnd, length=7, cmap = 'jet_r', linewidth =15)
        q = ax.quiver(x,y, u = uwnd[::10,::10], v= vwnd[::10,::10],
                  transform=ccrs.PlateCarree(), color='k',width=0.004, minlength=0.1)
        qk = ax.quiverkey(q, 1.1, 0.98, 20, r'20 m/s', labelpos='E')
    
    cbar = plt.colorbar(cf, orientation='horizontal', pad=0.05 , ax=ax, shrink = 1.0, aspect=40)
    cbar.ax.tick_params(labelsize=20)
    #cbar.set_label("W / m²", fontsize = 20, y = 0.3)
    
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=1, color='black', alpha=0.3, linestyle='--')
    gl.xlabels_top = False
    gl.ylabels_left = True
    gl.ylabels_right = False
    gl.ylines = True
    gl.xlines = True
    #gl.xlocator = mticker.FixedLocator(np.arange(-70, 50, 20)) #-70,50,20
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 20}
    gl.ylabel_style = {'size': 20}
    
    plt.tight_layout()
    ax.coastlines('50m')
    #ax.set_extent([-70, 20, -50, 10])
    ax.set_title(var, fontdict={'fontsize': 35}, loc = 'left', style ='oblique', weight='bold', pad = 10)
    ax.set_title('Média de ' + tempo[0] + ' ' + tempo[1], fontdict={'fontsize': 35}, loc = 'right', pad = 10)

In [ ]:
levels = [200, 500, 850]
variaveis1 = ['uwnd','vwnd','w','hgt']
variaveis2 = ['msl','ttr']
vortdiv = ['div', 'vort']

In [ ]:
figuraComp(var = 'vort', level = 200, periodoBase= tempoBase, periodo = tempoAnalise, title ='Comparativo da climatologia (1979 - 2019) e 2009' )

In [ ]:
for variavel in vortdiv:
    for cada in levels:
        figuraComp(var = variavel,level = cada, periodoBase = tempoBase, periodo = tempoAnalise, title = 'Comparativo da climatologia (1979 - 2019) e 2009')
        plt.savefig('D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/ZCAS/exercício2/' + variavel + str(cada) +'Comp.png')

In [ ]:
for variavel in variaveis1:
    for cada in levels:
        figuraComp(var = variavel,level = cada, periodoBase = tempoBase, periodo = tempoAnalise, title = 'Comparativo da climatologia (1979 - 2019) e 2009')
        plt.savefig('D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/ZCAS/exercício2/' + variavel + str(cada) +'Comp.png')

In [ ]:
for variavel in variaveis2:
    figuraComp(var = variavel,level = None, periodoBase = tempoBase, periodo = tempoAnalise, title = 'Comparativo da climatologia (1979 - 2019) e 2009')
    plt.savefig('D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/ZCAS/exercício2/' + variavel +'Comp.png')

# Anomalia

In [ ]:
def figuraAnom(var, level, periodoBase, periodo, title):
    fig, ax = plt.subplots(nrows=1, ncols=1, subplot_kw=dict(projection = ccrs.PlateCarree()), figsize=(32,26))
    fig.suptitle(title, style ='oblique', fontsize=34, weight='bold', y = 0.95, x = 0.505)
    
    plotAnom(ax, var, level, periodoBase, periodo)

In [ ]:
def plotAnom(ax, var, level, periodoBase, periodo):
    if var == 'div':
        lons, lats, valorBase, uwnd, vwnd = div(var, level, periodoBase)
        lons, lats, valorAnalise, uwnd, vwnd = div(var, level, periodo)
    elif var == 'vort':
        lons, lats, valorBase, uwnd, vwnd = vort(var, level, periodoBase)
        lons, lats, valorAnalise, uwnd, vwnd = div(var, level, periodo)
    else:
        lons, lats, valorBase, uwnd, vwnd = data(var, level, periodoBase)
        lons, lats, valorAnalise, uwnd, vwnd = div(var, level, periodo)
    
    anom = valorAnalise - valorBase
    
    #clevs = np.arange(11000, 13000, 150)
    cf = ax.contourf(lons, lats, anom, extend='both', cmap = 'RdBu_r', transform=ccrs.PlateCarree())
    
    if level != None:
        x = lons[::10]
        y = lats[::10]
        #brb = ax.barbs(x, y, uwnd, vwnd, length=7, cmap = 'jet_r', linewidth =15)
        q = ax.quiver(x,y, u = uwnd[::10,::10], v= vwnd[::10,::10],
                  transform=ccrs.PlateCarree(), color='k',width=0.002, minlength=0.1)
        qk = ax.quiverkey(q, 1.1, 0.98, 20, r'20 m/s', labelpos='E')
    
    
    cbar = plt.colorbar(cf, orientation='horizontal', pad=0.05 , ax=ax, shrink = 1.0, aspect=40)
    cbar.ax.tick_params(labelsize=20)
    #cbar.set_label("W / m²", fontsize = 20, y = 0.3)
    
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=1, color='black', alpha=0.3, linestyle='--')
    gl.xlabels_top = False
    gl.ylabels_left = True
    gl.ylabels_right = False
    gl.ylines = True
    gl.xlines = True
    #gl.xlocator = mticker.FixedLocator(np.arange(-70, 50, 20)) #-70,50,20
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 20}
    gl.ylabel_style = {'size': 20}
    
    plt.tight_layout()
    ax.coastlines('50m')
    #ax.set_extent([-70, 20, -50, 10])
    ax.set_title(var, fontdict={'fontsize': 35}, loc = 'right', style ='oblique', weight='bold', pad = 10)
    #ax.set_title('Anomalia de 2009', fontdict={'fontsize': 35}, loc = 'right', pad = 10)

In [ ]:
for variavel in variaveis2:
    figuraAnom(var = variavel, level = None, periodoBase = tempoBase, periodo = tempoAnalise, title='Anomalia de 2009 relativa à climatologia (1979-2019)')
    plt.savefig('D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/ZCAS/exercício2/' + variavel +'Anom.png')

In [ ]:
for variavel in variaveis1:
    for cada in levels:
        figuraAnom(var = variavel, level = cada, periodoBase = tempoBase, periodo = tempoAnalise, title='Anomalia de 2009 relativa à climatologia (1979-2019)')
        plt.savefig('D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/ZCAS/exercício2/' + variavel + str(cada) + 'Anom.png')

In [ ]:
for variavel in vortdiv:
    for cada in levels:
        figuraAnom(var = variavel, level = cada, periodoBase = tempoBase, periodo = tempoAnalise, title='Anomalia de 2009 relativa à climatologia (1979-2019)')
        plt.savefig('D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/ZCAS/exercício2/' + variavel + str(cada) + 'Anom.png')